In [1]:
if __name__ == '__main__':
    import matplotlib
    # Agg backend runs without a display
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
  
!pip install tensorflow-gpu 

  Using cached https://files.pythonhosted.org/packages/d0/33/376510eb8d6246f3c30545f416b2263eee461e40940c2a4413c711bdf62d/google_pasta-0.1.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/4e/35/11749bf99b2d4e3cceb4d55ca22590b0d7c2c62b9de38ac4a4a7f4687421/gast-0.2.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Using cached https://files.pythonhosted.org/package

1.14.0


In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


"""
Mask R-CNN
Train on the nuclei segmentation dataset from the
Kaggle 2018 Data Science Bowl
https://www.kaggle.com/c/data-science-bowl-2018/

Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------

Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:

    # Train a new model starting from ImageNet weights
    python3 Manga.py train --dataset=/path/to/dataset --subset=train --weights=imagenet

    # Train a new model starting from specific weights file
    python3 Manga.py train --dataset=/path/to/dataset --subset=train --weights=/path/to/weights.h5

    # Resume training a model that you had trained earlier
    python3 Manga.py train --dataset=/path/to/dataset --subset=train --weights=last

    # Generate submission file
    python3 Manga.py detect --dataset=/path/to/dataset --subset=train --weights=<last or /path/to/weights.h5>
"""



import os
import sys
import json
import datetime
import numpy as np
import skimage.io
from imgaug import augmenters as iaa


# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize

# Path to trained weights file
#COCO_WEIGHTS_PATH = os.path.join('/content/dataset/logs/manga20190730T1026', "mask_rcnn_manga_0433.h5")

COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_manga_2000.h5") #모델의 경로

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs") #필요없음

RESULTS_DIR = os.path.join(ROOT_DIR, "results/Manga/") #저장할 이미지의 경로


VAL_IMAGE_IDS = []# 여기에 이미지 디렉토리 이미지 이름 확장자 제외하고 리스트화


C:\Users\sanghoon\AssIstoon


Using TensorFlow backend.


In [2]:


# In[10]:


############################################################
#  Configurations
############################################################

class MangaConfig(Config):

    """Configuration for training on the Manga segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "Manga"
    IMAGE_RESIZE_MODE='pad64'
    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 1216
    GPU_COUNT = 1
    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2
    #LOSS_WEIGHTS = {
    #"rpn_class_loss": 0.0113,
    #"rpn_bbox_loss": 0.0464,
    #"mrcnn_class_loss": 0.0206,
    #"mrcnn_bbox_loss": 0.0157,
    #"mrcnn_mask_loss": 0.1544}
    '''
    
    rpn_class_loss: 0.0113 - rpn_bbox_loss: 0.0464 - mrcnn_class_loss: 0.0206 - mrcnn_bbox_loss: 0.0157 - mrcnn_mask_loss: 0.1544
    '''
    LEARNING_RATE=0.0001
    TRAIN_BN = False
    BACKBONE="resnet50"
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + Manga
    
    USE_MINI_MASK = True
    MINI_MASK_SHAPE = (112, 112)

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 8
    VALIDATION_STEPS = 5

    
    MAX_GT_INSTANCES=30
    # Don't exclude based on confidence. Since we have two classes
    # then 0.5 is the minimum anyway as it picks between Manga and BG
    DETECTION_MIN_CONFIDENCE = 0.9
    RPN_NMS_THRESHOLD = 0.9
    #BATCH_SIZE = 1

class MangaInferenceConfig(MangaConfig):
    IMAGE_RESIZE_MODE = "pad64"
    def __init__(self,dataset=None):
        
        self.h=2112
        self.w=1344

        self.IMAGE_CHANNEL_COUNT=3
        """Set values of computed attributes."""
        # Effective batch size
        self.BATCH_SIZE = 1
        self.IMAGE_SHAPE = np.array([self.h, self.w,
                self.IMAGE_CHANNEL_COUNT])
        self.IMAGE_META_SIZE = 1 + 3 + 3 + 4 + 1 + self.NUM_CLASSES
    # Set batch size to 1 to run one image at a time

    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    #len(VAL_IMAGE_IDS)
    #BATCH_SIZE = len(VAL_IMAGE_IDS)
    # Don't resize imager for inferencing
    
    DETECTION_MIN_CONFIDENCE = 0.9
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.9
    







In [24]:
# In[18]:



############################################################
#  Dataset
############################################################

class MangaDataset(utils.Dataset):
    def __init__(self):
        super(MangaDataset,self).__init__()
    

    def load_Manga(self,
                   dataset_dir,
                   subset,config=None): 
        """망가데이터셋의 서브셋 불러오기.

        dataset_dir: 데이터셋의 루트 디렉토리
        subset: 불러올 서브셋. 서브디렉토리의 이름,
                예를 들면 stage1_train, stage1_test, ...etc. 또는, 다음 중 하나:
                * train: stage1_train excluding validation images
                * val: VAL_IMAGE_IDS에 해당하는 validation 이미지
        """
        # 클래스를 추가한다
        # Naming the dataset Manga, and the class Manga
        self.add_class("Manga", 1, "sfx")
        self.add_class("Manga", 2, "text")
        self.add_class("Manga", 3, "face")

        # 어떤 subset? val or train?
        # "val": use hard-coded list above
        # "train": use data from stage1_train minus the hard-coded list above
        # else: use the data from the specified sub-directory
        #assert subset in ["train", "val", "stage1_train", "stage1_test", "stage2_test"]
        subset_dir = "train" if subset in ["train", "val"] else subset #수정필요
        #dataset_dir = 'C:/Users/sanghoon/AssIstoon/sfx_extract/dataset/'
        root_dir= dataset_dir+subset+'/'
        images_dir = root_dir+'images/'
        
        if subset=="train" :
            image_ids = [name[:-4] for name in os.listdir(images_dir)]  # 디렉토리 이름에서 ? 이미지 아이디를 받는다.
        elif subset=="val":
            image_ids=VAL_IMAGE_IDS
        else :
            image_ids = [name[:-4] for name in os.listdir(images_dir)]
        # Add images
        for idx,image_id in enumerate(image_ids):
            image_array=skimage.io.imread(images_dir+image_id+'.jpg')
            self.add_image("Manga",image_id,images_dir+'{}.jpg'.format(image_id),index=idx,shape=image_array.shape,image=image_array)
                ##padding 한 이미지
                
    def load_image(self, image_id):
        """Load the specified image and return a [H,W,3] Numpy array.
        """
        # Load image
        image = self.image_info[image_id]['image']
        # If grayscale. Convert to RGB for consistency.
        if image.ndim != 3:
            image = skimage.color.gray2rgb(image)
        # If has an alpha channel, remove it for consistency
        if image.shape[-1] == 4:
            image = image[..., :3]
            image_dtype = image.dtype
        # Default window (y1, x1, y2, x2) and default scale == 1.
        h, w = image.shape[:2]
        window = (0, 0, h, w)
        scale = 1
        padding = [(0, 0), (0, 0), (0, 0)]
        if h % 64 > 0:
            max_h = h - (h % 64) + 64
            top_pad = (max_h - h) // 2
            bottom_pad = max_h - h - top_pad
        else:
            top_pad = bottom_pad = 0
        # Width
        if w % 64 > 0:
            max_w = w - (w % 64) + 64
            left_pad = (max_w - w) // 2
            right_pad = max_w - w - left_pad
        else:
            left_pad = right_pad = 0
        padding = [(top_pad, bottom_pad), (left_pad, right_pad), (0, 0)]
        image = np.pad(image, padding, mode='constant', constant_values=0)
        window = (top_pad, left_pad, h + top_pad, w + left_pad)

        return image
    
    def load_mask(self, image_id):
        
        info = self.image_info[image_id] #image id에 대한 info를 얻어온다.

        # 이미지 경로로부터 마스크 디렉토리를 얻어온다.
        mask_dir = os.path.join(info['root'], "masks")
        # png이미지로부터 mask파일을 읽는다.
        mask_list = [f for f in os.listdir(mask_dir) if
                     f.split('_')[0] == info['id'].split('_')[0] + info['id'].split('_')[1] and f.endswith(".png")]

        mask=[]
        maskclasslist=[]
        for f in mask_list:
            maskclass=f.split('_')[1].split('-')[0]
            if maskclass == 'sfx':
                maskclasslist.append(1)
            elif maskclass == 'text':
                maskclasslist.append(2)
            elif maskclass == "face":
                maskclasslist.append(3)
            m = skimage.io.imread(os.path.join(mask_dir, f))
            m = m.sum(axis=2)

            #mask[m==True]=1
            mask.append(m)
        #mask=mask.astype(np.bool)
        #print(mask.shape)
        mask = np.stack(mask, axis=-1)


        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        return mask, np.array(maskclasslist, dtype=np.int32)

    def image_reference(self, image_id):
        """이미지의 경로를 반환한다."""
        info = self.image_info[image_id]
        if info["source"] == "Manga":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)



In [7]:
# In[19]:



############################################################
#  Training
############################################################

def train(model, dataset_dir=None):
    """Train the model."""
    # Training dataset.
    dataset_train = MangaDataset()
    dataset_train.load_Manga(dataset_dir,"train")
    dataset_train.prepare() #dataset 클래스의 참조 필요

    # Validation dataset
    dataset_val = MangaDataset()
    dataset_val.load_Manga(dataset_dir, "val")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        #iaa.OneOf([iaa.Affine(rotate=90),
        #           iaa.Affine(rotate=180),
        #           iaa.Affine(rotate=270)]),
        iaa.Multiply((0.8, 1.5)),
        iaa.GaussianBlur(sigma=(0.0, 5.0))
    ])#이건 뭐하는걸까? image augmentation인걸 보니 전처리를 해주는가보다.

    # *** 수정이 필요하면 맞게 수정하세요 ***

    # If starting from imagenet, train heads only for a bit
    # since they have random weights
    
    print("Train network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE, #config 참조 필요 혹은 직접 러닝레이트 지정
                epochs=20,
                augmentation=augmentation,
                layers='heads')

    
    print("Train all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=100,
                augmentation=augmentation,
                layers='all')




In [8]:
# In[20]:



############################################################
#  RLE Encoding
############################################################
#RLE란 무엇인가? 간단한 비손실 압축 방법으로 데이터에서 같은 값이 연속해서 나타나는 것을 그 개수와 반복되는 값만으로 표시하는 방법.
def rle_encode(mask): #mask를 RLE 인코딩 하나보다
    """Encodes a mask in Run Length Encoding (RLE).
    Returns a string of space-separated values.
    """
    assert mask.ndim == 2, "mask는 다음과 같은 모양이여야 한다. [Height, Width]"
    # Flatten it column wise 
    m = mask.T.flatten()
    # Compute gradient. Equals 1 or -1 at transition points
    g = np.diff(np.concatenate([[0], m, [0]]), n=1)
    # 1-based indicies of transition points (where gradient != 0)
    rle = np.where(g != 0)[0].reshape([-1, 2]) + 1
    # Convert second index in each pair to lenth
    rle[:, 1] = rle[:, 1] - rle[:, 0]
    return " ".join(map(str, rle.flatten()))


def rle_decode(rle, shape): #압축한 mask 인코딩 데이터를 디코딩한다.
    """Decodes an RLE encoded list of space separated
    numbers and returns a binary mask."""
    rle = list(map(int, rle.split()))
    rle = np.array(rle, dtype=np.int32).reshape([-1, 2])
    rle[:, 1] += rle[:, 0]
    rle -= 1
    mask = np.zeros([shape[0] * shape[1]], np.bool)
    for s, e in rle:
        assert 0 <= s < mask.shape[0]
        assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {}".format(shape, s, e)
        mask[s:e] = 1
    # Reshape and transpose
    mask = mask.reshape([shape[1], shape[0]]).T
    return mask


def mask_to_rle(image_id, mask, scores): #
    "Encodes instance masks to submission format."
    assert mask.ndim == 3, "Mask must be [H, W, count]"
    # If mask is empty, return line with image ID only
    if mask.shape[-1] == 0:
        return "{},".format(image_id)
    # 겹치는 mask 제거
    # Multiply each instance mask by its score order
    # then take the maximum across the last dimension #점수 제일 높은걸 고른다는 듯.
    order = np.argsort(scores)[::-1] + 1  # 1-based descending
    mask = np.max(mask * np.reshape(order, [1, 1, -1]), -1)
    # Loop over instance masks
    lines = []
    for o in order:
        m = np.where(mask == o, 1, 0)
        # Skip if empty
        if m.sum() == 0.0:
            continue
        rle = rle_encode(m)
        lines.append("{}, {}".format(image_id, rle))
    return "\n".join(lines)



In [26]:

# In[21]:



############################################################
#  Detection
############################################################

def detect(model, dataset_dir, subset):
    import matplotlib.pyplot as plt
    """Run detection on images in the given directory."""
    print("Running on {}".format(dataset_dir))

    # Create directory
    if not os.path.exists(RESULTS_DIR):
        os.makedirs(RESULTS_DIR)
    submit_dir = "submit_{:%Y%m%dT%H%M%S}".format(datetime.datetime.now())
    submit_dir = os.path.join(RESULTS_DIR, submit_dir)
    os.makedirs(submit_dir)

    # Read dataset
    dataset = MangaDataset()
    dataset.load_Manga(dataset_dir, subset) #불러옴
    dataset.prepare()
    # Load over images
    submission = []
    for image_id in dataset.image_ids:
        # Load image and run detection
        image = dataset.load_image(image_id)
        print(type(image))
        # Detect objects
        r = model.detect([image], verbose=0)[0]
        # Encode image to RLE. Returns a string of multiple lines
        source_id = dataset.image_info[image_id]["id"]
        rle = mask_to_rle(source_id, r["masks"], r["scores"])
        submission.append(rle)
        # Save image with masks
        visualize.display_instances(
            image, r['rois'], r['masks'], r['class_ids'],
            dataset.class_names, r['scores'],
            show_bbox=False, show_mask=True,
            title="Predictions")
        plt.savefig("{}/{}.png".format(submit_dir, dataset.image_info[image_id]["id"]))

    # Save to csv file
    #submission = "ImageId,EncodedPixels\n" + "\n".join(submission)
    file_path = os.path.join(submit_dir, "submit.csv")
    with open(file_path, "w") as f:
        f.write(submission)
    print("Saved to ", submit_dir)

In [27]:

if __name__ == '__main__':
    import argparse
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    # Parse command line arguments
    '''
    parser = argparse.ArgumentParser(
        description='Mask R-CNN for AssIstoon counting and segmentation')
    parser.add_argument("command",
                        metavar="<command>",
                        help="'train' or 'detect'")
    parser.add_argument('--dataset', required=False,
                        metavar="/path/to/dataset/",
                        help='Root directory of the dataset')
    parser.add_argument('--weights', required=True,
                        metavar="/path/to/weights.h5", #수정 필요? 
                        help="Path to weights .h5 file or 'coco'")
    parser.add_argument('--logs', required=False,
                        default=DEFAULT_LOGS_DIR,
                        metavar="/path/to/logs/",
                        help='Logs and checkpoints directory (default=logs/)')
    parser.add_argument('--subset', required=False,
                        metavar="Dataset sub-directory",
                        help="Subset of dataset to run prediction on")
    args = parser.parse_args()

    '''

    '''dataset_train = MangaDataset()
    dataset_train.load_Manga()
    dataset_train.prepare()
    print("Image Count: {}".format(len(dataset_train.image_ids)))
    print("Class Count: {}".format(dataset_train.num_classes))
    for i, info in enumerate(dataset_train.class_info):
        print("{:3}. {:50}".format(i, info['name']))

    image_ids = np.random.choice(dataset_train.image_ids, 4)
    print(image_ids)

    for image_id in image_ids:

        image = dataset_train.load_image(image_id)
        mask, class_ids = dataset_train.load_mask(image_id)
        visualize.display_top_masks(image, mask, class_ids,dataset_train.class_names, limit=1)
'''
    command="train"
    command='detect'
    # Validate arguments
    #if args.command == "train":
        #assert args.dataset, "Argument --dataset is required for training"
    #elif args.command == "detect":
    #    assert args.subset, "Provide --subset to run prediction on"

    #print("Weights: ", args.weights)
    #print("Dataset: ", args.dataset)
        #if args.subset:
    #print("Subset: ", args.subset)
    #print("Logs: ", args.logs)

    # Configurations
    #if args.command == "train":
    if command=='train':
        config = MangaConfig()
    else:
        config = MangaInferenceConfig()
    config.display()

    # Create model
    #if args.command == "train":
    if command=="train":
        model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=
                                "D:/logs")
    else:
        model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=
                                 "D:/logs/")
    weights="coco"
    # Select weights file to load
    #if args.weights.lower() == "coco":
    if weights=="coco":
        weights_path = COCO_WEIGHTS_PATH
        # Download weights file
        if not os.path.exists(weights_path):
            utils.download_trained_weights(weights_path)
    elif weights.lower() == "last":
        # Find last trained weights
        weights_path = model.find_last()
    elif weights.lower() == "imagenet":
        # Start from ImageNet trained weights
        weights_path = model.get_imagenet_weights()
    else:
        weights_path = args.weights

    # Load weights
    print("Loading weights ", weights_path)
    if weights.lower() == "coco":
        # Exclude the last layers because they require a matching
        # number of classes
        model.load_weights(weights_path, by_name=True
            #, exclude=[
            #"mrcnn_class_logits",
            #"mrcnn_bbox_fc",
            #"mrcnn_bbox",
            #"mrcnn_mask"
            #]
        )

    else:
        model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits",
            "mrcnn_bbox_fc",
            "mrcnn_bbox",
            "mrcnn_mask"
            ])

    dataset='D:/sfx_extract_adam_plus/dataset/'
    if command == "train":
        train(model, dataset)
    elif command == "detect":
        detect(model, dataset, 'new')
    else:
        print("'{}' is not recognized. "
              "Use 'train' or 'detect'".format(args.command))

    # Train or evaluate


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1216
IMAGE_META_SIZE                16
IMAGE_MIN_DIM                  768
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [2112 1344    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

TypeError: 'NoneType' object is not subscriptable

In [0]:


from google.colab import files
path='/content/dataset/logs/manga20190728T0623/'
filelist=[path+name for name in os.listdir(path)]
print(filelist)
for file in filelist:
  files.download(file) 